### generic model for face detection (based on an object detection model)
### model
- `VGGIG` - predefined model for object detection
- input 
    - image [200 x 200]
- output 
    - [0,1] - 0 if face is not present on the image, 1 otherwise
    - x,y,w,h - coordinates of the face on the image
### dictionary
- bounding box - box that marks area where our object is present (in our case face)
### libraries 
- `labelme` - Library that contains method that we will use to label data


In [1]:
from pathlib import Path # to get the path of the current file
import time # to measure time
import uuid # for unique id
import cv2

# collect data

In [2]:
DATA_PATH = Path.cwd() / 'data' / 'images' # path to the data folder
IMAGES_NUMBER = 0
DATA_PATH.mkdir(parents=True, exist_ok=True)
DATA_PATH

WindowsPath('C:/Users/miskibin/Desktop/studies/POIWK_laby/model/data/images')

In [3]:
capture = cv2.VideoCapture(0) # 0 is the default camera
for num in range(IMAGES_NUMBER):
    print(f'Image {num}/{IMAGES_NUMBER}')
    ret, frame = capture.read()
    if ret:
        img_name = str(Path(DATA_PATH / f'{uuid.uuid4()}.jpg').resolve())
        cv2.imwrite(img_name, frame)
    cv2.imshow('frame', frame)
    time.sleep(.5) 
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
capture.release()
cv2.destroyAllWindows()